# NBA Player Performance Analysis - Quadrant Plot

This notebook analyzes NBA player performance data to create a quadrant plot that visualizes offensive and defensive capabilities. We'll develop two metrics to represent offensive and defensive performance, then plot players on a coordinate system to identify different player types.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [ ]:
# Load the NBA dataset
nba_data = pd.read_csv('NBA.csv', encoding="latin1")

# Display the first few rows to understand the data
nba_data.head()


In [ ]:
# Check for missing values
nba_data.isnull().sum()


In [ ]:
# Get basic statistics of the dataset
nba_data.describe()


## Data Preprocessing

The dataset contains multiple entries for some players who were traded mid-season. We'll handle this by aggregating their statistics.


In [ ]:
# Handle duplicate players (due to mid-season trades)
# Group by player name and aggregate stats (taking the mean for simplicity)
player_stats = nba_data.groupby('Player').agg({
    'G': 'sum',  # Sum games played across teams
    'MP': 'mean',  # Average minutes per game
    'PTS': 'mean',  # Average points per game
    'FG%': 'mean',  # Field goal percentage
    '3P%': 'mean',  # 3-point percentage
    'FT%': 'mean',  # Free throw percentage
    'TRB': 'mean',  # Average total rebounds
    'AST': 'mean',  # Average assists
    'STL': 'mean',  # Average steals
    'BLK': 'mean',  # Average blocks
    'TOV': 'mean',  # Average turnovers
    'PF': 'mean',   # Average personal fouls
    'eFG%': 'mean', # Effective field goal percentage
    'ORB': 'mean',  # Offensive rebounds
    'DRB': 'mean',  # Defensive rebounds
}).reset_index()

# Filter players with minimum games played to ensure meaningful stats
min_games = 10
player_stats = player_stats[player_stats['G'] >= min_games]


## Creating Performance Metrics

We'll create two metrics:
1. **Offensive Performance**: Combining points, assists, offensive rebounds, and shooting efficiency
2. **Defensive Performance**: Combining defensive rebounds, steals, blocks, and inverse of personal fouls


In [ ]:
# Create offensive performance metric
player_stats['offensive_score'] = (
    player_stats['PTS'] * 0.4 +  # Points scored
    player_stats['AST'] * 0.3 +  # Assists (creating scoring opportunities)
    player_stats['ORB'] * 0.1 +  # Offensive rebounds (second chance points)
    player_stats['eFG%'] * 100 * 0.2  # Shooting efficiency
)

# Create defensive performance metric
player_stats['defensive_score'] = (
    player_stats['DRB'] * 0.3 +  # Defensive rebounds
    player_stats['STL'] * 0.3 +  # Steals
    player_stats['BLK'] * 0.3 +  # Blocks
    (6 - player_stats['PF']) * 0.1  # Inverse of personal fouls (6 is max fouls)
)

# Standardize the metrics (convert to z-scores)
scaler = StandardScaler()
player_stats[['offensive_z', 'defensive_z']] = scaler.fit_transform(
    player_stats[['offensive_score', 'defensive_score']]
)


## Creating the Quadrant Plot

We'll create a quadrant plot with:
- Offensive performance on the x-axis
- Defensive performance on the y-axis
- Quadrant boundaries at the origin (0,0)
- Different colors for players based on minutes played
- Tooltips showing player information


In [ ]:
# Create the quadrant plot
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter(
    x=player_stats['offensive_z'],
    y=player_stats['defensive_z'],
    mode='markers',
    marker=dict(
        size=10,
        color=player_stats['MP'],  # Color by minutes played
        colorscale='Viridis',
        colorbar=dict(title='Minutes Per Game'),
        opacity=0.8
    ),
    text=player_stats['Player'],
    hovertemplate='<b>%{text}</b><br>' +
                  'Offensive Score: %{x:.2f}<br>' +
                  'Defensive Score: %{y:.2f}<br>' +
                  'Minutes Per Game: %{marker.color:.1f}',
    name=''
))

# Add quadrant lines
fig.add_shape(
    type="line", x0=-4, y0=0, x1=4, y1=0,
    line=dict(color="black", width=1, dash="dash")
)
fig.add_shape(
    type="line", x0=0, y0=-4, x1=0, y1=4,
    line=dict(color="black", width=1, dash="dash")
)

# Add quadrant labels
fig.add_annotation(x=2, y=2, text="Elite Two-Way Players",
                  showarrow=False, font=dict(size=14))
fig.add_annotation(x=2, y=-2, text="Offensive Specialists",
                  showarrow=False, font=dict(size=14))
fig.add_annotation(x=-2, y=2, text="Defensive Specialists",
                  showarrow=False, font=dict(size=14))
fig.add_annotation(x=-2, y=-2, text="Below Average",
                  showarrow=False, font=dict(size=14))

# Update layout
fig.update_layout(
    title='NBA Player Performance Quadrant Analysis',
    xaxis_title='Offensive Performance (z-score)',
    yaxis_title='Defensive Performance (z-score)',
    xaxis=dict(range=[-3, 3]),
    yaxis=dict(range=[-3, 3]),
    width=900,
    height=700,
    template='plotly_white'
)

# Show the plot
fig.show()


## Alternative Approach: PCA-based Metrics

As an alternative approach, we can use Principal Component Analysis (PCA) to create our offensive and defensive metrics. This allows the data to determine which statistics are most important for each dimension.


In [ ]:
# Select offensive stats
offensive_stats = ['PTS', 'AST', 'ORB', 'eFG%', 'FG%', '3P%', 'FT%']
# Select defensive stats
defensive_stats = ['DRB', 'STL', 'BLK', 'PF']

# Handle missing values for PCA
for col in offensive_stats + defensive_stats:
    if player_stats[col].isnull().any():
        player_stats[col] = player_stats[col].fillna(player_stats[col].mean())

# Apply PCA for offensive metrics
pca_offensive = PCA(n_components=1)
offensive_pca = pca_offensive.fit_transform(StandardScaler().fit_transform(player_stats[offensive_stats]))
player_stats['offensive_pca'] = offensive_pca

# Apply PCA for defensive metrics
pca_defensive = PCA(n_components=1)
defensive_pca = pca_defensive.fit_transform(StandardScaler().fit_transform(player_stats[defensive_stats]))
player_stats['defensive_pca'] = defensive_pca

# Ensure positive correlation with performance (PCA might flip the sign)
if pca_offensive.components_[0][0] < 0:  # If PTS has negative loading
    player_stats['offensive_pca'] = -player_stats['offensive_pca']
if pca_defensive.components_[0][1] < 0:  # If STL has negative loading
    player_stats['defensive_pca'] = -player_stats['defensive_pca']


In [ ]:
# Create the PCA-based quadrant plot
fig_pca = go.Figure()

# Add scatter plot with PCA metrics
fig_pca.add_trace(go.Scatter(
    x=player_stats['offensive_pca'],
    y=player_stats['defensive_pca'],
    mode='markers',
    marker=dict(
        size=10,
        color=player_stats['MP'],
        colorscale='Viridis',
        colorbar=dict(title='Minutes Per Game'),
        opacity=0.8
    ),
    text=player_stats['Player'],
    hovertemplate='<b>%{text}</b><br>' +
                  'Offensive Score (PCA): %{x:.2f}<br>' +
                  'Defensive Score (PCA): %{y:.2f}<br>' +
                  'Minutes Per Game: %{marker.color:.1f}',
    name=''
))

# Add quadrant lines
fig_pca.add_shape(
    type="line", x0=-4, y0=0, x1=4, y1=0,
    line=dict(color="black", width=1, dash="dash")
)
fig_pca.add_shape(
    type="line", x0=0, y0=-4, x1=0, y1=4,
    line=dict(color="black", width=1, dash="dash")
)

# Add quadrant labels
fig_pca.add_annotation(x=2, y=2, text="Elite Two-Way Players",
                      showarrow=False, font=dict(size=14))
fig_pca.add_annotation(x=2, y=-2, text="Offensive Specialists",
                      showarrow=False, font=dict(size=14))
fig_pca.add_annotation(x=-2, y=2, text="Defensive Specialists",
                      showarrow=False, font=dict(size=14))
fig_pca.add_annotation(x=-2, y=-2, text="Below Average",
                      showarrow=False, font=dict(size=14))

# Update layout
fig_pca.update_layout(
    title='NBA Player Performance Quadrant Analysis (PCA-based)',
    xaxis_title='Offensive Performance (PCA)',
    yaxis_title='Defensive Performance (PCA)',
    xaxis=dict(range=[-3, 3]),
    yaxis=dict(range=[-3, 3]),
    width=900,
    height=700,
    template='plotly_white'
)

# Show the PCA-based plot
fig_pca.show()


## Conclusion

The quadrant plot provides a clear visualization of player performance across both offensive and defensive dimensions. Players are categorized into four groups:

1. **Elite Two-Way Players** (top-right quadrant): These players excel in both offensive and defensive aspects of the game.
2. **Offensive Specialists** (bottom-right quadrant): These players are strong offensively but below average defensively.
3. **Defensive Specialists** (top-left quadrant): These players are strong defensively but below average offensively.
4. **Below Average** (bottom-left quadrant): These players are below average in both offensive and defensive metrics.

The color gradient based on minutes played adds another dimension to the analysis, showing which players receive more playing time. Generally, players who excel in both dimensions (top-right quadrant) tend to receive more minutes, as indicated by the darker colors in that region.

This visualization allows coaches, analysts, and fans to quickly identify player strengths and weaknesses, and can be useful for team building, matchup analysis, and player development planning.